In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier


In [2]:
df=pd.read_csv('adult.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [4]:
df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [5]:
df.replace('?',np.nan,inplace=True)

In [6]:
df.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

In [7]:
X=df.drop('income',axis=1)
y=df['income']


cat_col=X.select_dtypes(include='object').columns
num_col=X.select_dtypes(exclude='object').columns


In [8]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
Num_pipe=Pipeline(
       steps=[
           ('simple',SimpleImputer(strategy='mean')),
           ('scalar',StandardScaler())
       ]
)

cat_pipe=Pipeline(
    steps=[
        ('cat',SimpleImputer(strategy='most_frequent')),
        ('one',OneHotEncoder())

    ]
)

In [10]:
pre=ColumnTransformer(
    transformers=[
        ('pre',Num_pipe,num_col),
        ('pre1',cat_pipe,cat_col)
    ]
)

In [11]:
pip=Pipeline(
    steps=[
        ('preprocessing',pre)
        ,('model',DecisionTreeClassifier(random_state=42))
    ],
)

In [12]:
pip

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('pre', ...), ('pre1', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [13]:
pip.fit(Xtrain,ytrain)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('pre', ...), ('pre1', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
pip.score(Xtrain,ytrain)

0.9999616093366094

In [15]:
pip.score(Xtest,ytest)

0.8140641793336404

In [16]:
bag=Pipeline(
    steps=[
        ('preprocessing',pre)
        ,('model',RandomForestClassifier(n_estimators=10))
    ],
)

In [17]:
bag.fit(Xtrain,ytrain)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('pre', ...), ('pre1', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [18]:
bag.score(Xtest,ytest)

0.8423153692614771

In [19]:
bag.score(Xtrain,ytrain)

0.9875230343980343

In [20]:
params={
    'model__criterion':['entropy','gini'],
    'model__min_samples_split':[2,3,5,7,9],
    'model__max_depth':[5,10,50,100],
    'model__min_samples_leaf':[2,3,5,7,9]
}

In [21]:
grid=GridSearchCV(bag,params,cv=5,n_jobs=-1)

In [22]:
grid.fit(Xtrain,ytrain)

c:\Users\DELL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.war

,estimator,Pipeline(step...imators=10))])
,param_grid,"{'model__criterion': ['entropy', 'gini'], 'model__max_depth': [5, 10, ...], 'model__min_samples_leaf': [2, 3, ...], 'model__min_samples_split': [2, 3, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('pre', ...), ('pre1', ...)]"


In [23]:
grid.score(Xtrain,ytrain)

0.8263590294840295

In [24]:
grid.score(Xtest,ytest)

0.8254260709350529

In [25]:
grid.best_params_

{'model__criterion': 'entropy',
 'model__max_depth': 5,
 'model__min_samples_leaf': 2,
 'model__min_samples_split': 2}

In [26]:
from xgboost import XGBClassifier

In [ ]:
boost=XGBClassifier()
label=label
boost.fit(Xtrain,ytrain)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got ['<=50K' '>50K']